In [1]:
import trimesh
import networkx
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from tqdm import tqdm
from torch.utils.data import Dataset
from pathlib import Path

In [2]:
class STLDataset(Dataset):

    def __init__(self, root_path, tensor_size):
        self.stl_path = root_path
        self.stl_list = sorted(self._get_filenames(self.stl_path))
        self.tensor_size = tensor_size

    def __getitem__(self, idx):
        mesh = trimesh.load_mesh(self.stl_list[idx], force='mesh')
        adj = torch.from_numpy(networkx.adjacency_matrix(trimesh.graph.vertex_adjacency_graph(mesh)).toarray())
        adj = adj.float()
        triangles_num = torch.tensor(mesh.faces.shape[0])
        triangle_vertices_coords = self.fix_size_coords(torch.from_numpy(mesh.vertices))
        adj = self.fix_size_adj(adj, tensor_size=self.tensor_size)
        return adj, triangles_num, triangle_vertices_coords

    def __len__(self):
        return len(self.stl_list)
    
    @staticmethod
    def _get_filenames(path):
        return [f for f in path.iterdir() if f.is_file()]

    @staticmethod
    def fix_size_adj(input_tensor, tensor_size=42):
        if input_tensor.shape[0] < tensor_size:
            zeros = torch.zeros(input_tensor.shape[0], tensor_size - input_tensor.shape[0])
            tensor = torch.cat([input_tensor, zeros], dim=1)
    
            zeros = torch.zeros(tensor_size - input_tensor.shape[0], tensor_size)
            tensor = torch.cat([tensor, zeros], dim=0)
            return tensor
        elif input_tensor.shape[0] > tensor_size:
            return input_tensor[:tensor_size, :tensor_size]
        else:
            return input_tensor

    @staticmethod
    def fix_size_coords(input_tensor, tensor_size=42):
        if input_tensor.shape[0] < tensor_size:
            zeros = torch.zeros(tensor_size - input_tensor.shape[0], input_tensor.shape[1])
            tensor = torch.cat([input_tensor, zeros], dim=0)

            return tensor
        elif input_tensor.shape[0] > tensor_size:
            return input_tensor[:tensor_size]
        else:
            return input_tensor

In [3]:
input_dir = "../data/Thingi10K/models"
ADJACENCY_MATRIX_SIZE = 100

dataset = STLDataset(Path(input_dir), tensor_size=ADJACENCY_MATRIX_SIZE)

In [4]:
batch_size = 8
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, pin_memory=True
)

### Initialise model.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
# class TriangleNet(nn.Module):
#     """
#     Regression model for predicting number of triangles
#     based on the given adjacency matrix.

#     adj_size: size of the adjacency matrix.
#     """

#     def __init__(self, adj_size: int = 42):
#         super(TriangleNet, self).__init__()

#         # TODO: fix too many neurons
#         input_neurons = adj_size * adj_size
#         hidden_s = input_neurons // 2

#         self.layers = nn.Sequential(
#             nn.Linear(input_neurons, hidden_s),
#             nn.ReLU(),
#             nn.Linear(hidden_s, 1)
#         )

#     def forward(self, x: torch.tensor):
#         x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
#         x = self.layers(x)

#         # round output
#         # x = torch.round(x)
#         # x = x.type(torch.long)
#         return x.flatten()

# Params

In [7]:
# EPOCHS = 1
# triangle_net = TriangleNet(ADJACENCY_MATRIX_SIZE).to(device)
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.Adam(triangle_net.parameters(), lr=0.001)
# triangle_net

### Train loop

In [8]:
# def train_one_epoch():
#     running_loss = 0.
#     last_loss = 0.

#     # Here, we use enumerate(training_loader) instead of
#     # iter(training_loader) so that we can track the batch
#     # index and do some intra-epoch reporting
#     for i, (adj, triangles_num, triangles_coords) in enumerate(data_loader):
#         # Every data instance is an input + label pair
#         triangles_num = triangles_num.type(dtype=torch.float32)

#         # Zero your gradients for every batch!
#         optimizer.zero_grad()

#         # print(adj)
#         # Make predictions for this batch
#         outputs = triangle_net(adj)

#         # Compute the loss and its gradients
#         loss = loss_fn(outputs, triangles_num)
#         loss.backward()

#         # Adjust learning weights
#         optimizer.step()

#         # Gather data and report
#         running_loss += loss.item()
#         if i % 1000 == 999:
#             last_loss = running_loss / 1000 # loss per batch
#             print('  batch {} loss: {}'.format(i + 1, last_loss))
#             running_loss = 0.

#     return last_loss

In [9]:
# best_vloss = 1_000_000.

# for epoch in tqdm(range(EPOCHS)):
#     print('EPOCH {}:'.format(epoch + 1))

#     # Make sure gradient tracking is on, and do a pass over the data
#     triangle_net.train(True)
#     avg_loss = train_one_epoch()

#     print("Train loss: ", avg_loss)

    # running_vloss = 0.0
    # # Set the model to evaluation mode, disabling dropout and using population
    # # statistics for batch normalization.
    # model.eval()

    # # Disable gradient computation and reduce memory consumption.
    # with torch.no_grad():
    #     for i, vdata in enumerate(validation_loader):
    #         vinputs, vlabels = vdata
    #         voutputs = model(vinputs)
    #         vloss = loss_fn(voutputs, vlabels)
    #         running_vloss += vloss

    # avg_vloss = running_vloss / (i + 1)
    # print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # # Track best performance, and save the model's state
    # if avg_vloss < best_vloss:
    #     best_vloss = avg_vloss
    #     model_path = 'triangle_net_{}'.format(epoch)
    #     torch.save(triangle_net.state_dict(), model_path)

# Create mesh from adj and coordinates

In [13]:
# mesh = trimesh.load_mesh("../data/Thingi10K/models/0.stl", force='mesh')
# adj_matrix = networkx.adjacency_matrix(trimesh.graph.vertex_adjacency_graph(mesh)).toarray()

adj_matrix, triangles_num, triangles_vert_coords = next(iter(data_loader))
adj_matrix, triangles_num, triangles_vert_coords = adj_matrix[3], triangles_num[3], triangles_vert_coords[3]

# vertices = mesh.vertices
# adj = adj.float()
# triangles_num = torch.tensor(mesh.faces)
print(adj_matrix.shape)
print(triangles_num)
# print(vertices.shape)

torch.Size([100, 100])
tensor(920)


In [14]:
# find all triangles in the adjacency matrix
triangles = []
for i in range(len(adj_matrix)):
   for j in range(i+1, len(adj_matrix)):
       if adj_matrix[i, j] == 1:
           for k in range(j+1, len(adj_matrix)):
               if adj_matrix[j, k] == 1 and adj_matrix[k, i] == 1:
                  triangles.append([i, j, k])
triangles

[[0, 1, 2],
 [0, 1, 3],
 [0, 2, 15],
 [0, 3, 16],
 [0, 15, 16],
 [1, 2, 14],
 [1, 3, 5],
 [1, 5, 27],
 [1, 14, 28],
 [1, 27, 28],
 [2, 14, 31],
 [2, 15, 31],
 [3, 4, 5],
 [3, 4, 17],
 [3, 16, 17],
 [4, 5, 13],
 [4, 6, 7],
 [4, 6, 13],
 [4, 7, 18],
 [4, 17, 18],
 [5, 13, 26],
 [5, 26, 27],
 [6, 7, 9],
 [6, 9, 25],
 [6, 13, 25],
 [7, 8, 9],
 [7, 8, 19],
 [7, 18, 19],
 [8, 9, 10],
 [8, 10, 20],
 [8, 19, 20],
 [9, 10, 12],
 [9, 12, 24],
 [9, 24, 25],
 [10, 11, 12],
 [10, 11, 21],
 [10, 20, 21],
 [11, 12, 22],
 [11, 21, 32],
 [11, 22, 33],
 [11, 32, 58],
 [11, 33, 58],
 [12, 22, 23],
 [12, 23, 24],
 [13, 25, 35],
 [13, 26, 35],
 [14, 28, 30],
 [14, 30, 31],
 [15, 16, 44],
 [15, 31, 40],
 [15, 40, 42],
 [15, 42, 43],
 [15, 42, 44],
 [15, 43, 44],
 [16, 17, 46],
 [16, 44, 45],
 [16, 45, 46],
 [17, 18, 48],
 [17, 46, 47],
 [17, 47, 48],
 [18, 19, 50],
 [18, 48, 49],
 [18, 49, 50],
 [19, 20, 51],
 [19, 50, 51],
 [20, 21, 53],
 [20, 51, 52],
 [20, 52, 53],
 [21, 32, 55],
 [21, 53, 54],
 [21, 54,

In [15]:
import trimesh

# create faces
faces = np.array(triangles)

# create a trimesh
mesh = trimesh.Trimesh(vertices=[], faces=faces)
mesh

<trimesh.Trimesh(vertices.shape=(0, 3), faces.shape=(171, 3))>